In [ ]:
from skimage.io import imread, imshow, concatenate_images
from skimage import color
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split
from skimage.exposure import equalize_adapthist
import os
import random
from tqdm import tqdm_notebook, tnrange
import numpy as np
# h5py to read the data-set
import h5py
# matplotlob for plotting
import matplotlib.pyplot as plt
# tensorboard
from keras.callbacks import TensorBoard
from keras.models import Model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.convolutional import Conv2D, Conv2DTranspose,UpSampling2D
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
# Set some parameters
im_width = 512
im_height = 512
border = 5
path_train = 'PATH/TO/THE/TRAIN/DATA' 
path_val = 'PATH/TO/THE/VALIDATION/DATA'
path_test = 'PATH/TO/THE/TEST/DATA'

In [ ]:
# Get and resize train images and masks
def get_data(path, train=True):
    ids = next(os.walk(path + "images"))[2]
    X = np.zeros((len(ids), im_height, im_width, 1), dtype=np.float32)
    if train:
        y = np.zeros((len(ids), im_height, im_width, 1), dtype=np.float32)
    print('Getting and resizing images ... ')
    for n, id_ in tqdm_notebook(enumerate(ids), total=len(ids)):
        # Load images
        img = load_img(path + '/images/' + id_, grayscale=True)
        x_img = img_to_array(img)
        x_img = resize(x_img, (512, 512, 1), mode='constant', preserve_range=True)

        # Load masks
        if train:
            mask = img_to_array(load_img(path + '/masks/' + id_, grayscale=True))
            mask = resize(mask, (512, 512, 1), mode='constant', preserve_range=True)

        # Save images
        X[n, ..., 0] = x_img.squeeze() / 255
        if train:
            y[n] = mask / 255
    print('Done!')
    if train:
        return X, y
    else:
        return X

In [ ]:
train_data, train_labels = get_data(path_train, train=True)

In [ ]:
val_data, val_labels = get_data(path_val, train=True)

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.subplot(221)
plt.imshow(train_data[0,:,:,0], cmap=plt.get_cmap('gray'))
plt.subplot(222)
plt.imshow(train_labels[0, :, :, 0], cmap=plt.get_cmap('gray'))
plt.subplot(223)
plt.imshow(train_data[10, :, :, 0], cmap=plt.get_cmap('gray'))
plt.subplot(224)
plt.imshow(train_labels[10, :, :, 0], cmap=plt.get_cmap('gray'))
# show the plot
plt.show()

In [ ]:
#Each block of U-net architecture consist of two Convolution layers
# These two layers are written in a function to make our code clean
def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    # second layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    return x

In [ ]:
# The u-net architecture consists of contracting and expansive paths which
# shrink and expands the inout image respectivly. 
# Output image have the same size of input image
def get_unet(input_img, n_filters, batchnorm=True):
    # contracting path
    c1 = conv2d_block(input_img, n_filters=n_filters*4, kernel_size=3) #The first block of U-net
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = conv2d_block(p1, n_filters=n_filters*8, kernel_size=3)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = conv2d_block(p2, n_filters=n_filters*16, kernel_size=3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = conv2d_block(p3, n_filters=n_filters*32, kernel_size=3)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
    
    c5 = conv2d_block(p4, n_filters=n_filters*64, kernel_size=3)
    
    # expansive path
    u6 = Conv2DTranspose(n_filters*32, (3, 3), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = conv2d_block(u6, n_filters=n_filters*32, kernel_size=3)

    u7 = Conv2DTranspose(n_filters*16, (3, 3), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = conv2d_block(u7, n_filters=n_filters*16, kernel_size=3)

    u8 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = conv2d_block(u8, n_filters=n_filters*8, kernel_size=3)

    u9 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = conv2d_block(u9, n_filters=n_filters*4, kernel_size=3)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

In [ ]:
# For a segmentation task, it is suggested to use dice coefficient instead of accuracy
def dice_coefficient(y_true, y_pred):
    eps = 1e-6
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection) / (K.sum(y_true_f * y_true_f) + K.sum(y_pred_f * y_pred_f) + eps)

In [ ]:
# Creating and Compiling the model
input_img = Input((train_data.shape[1], train_data.shape[2], 1), name='img')
model = get_unet(input_img, n_filters=2, batchnorm=True)

model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=[dice_coefficient])
model.summary()

In [ ]:
# saving the log and show it by tensorboard
NAME='U-Net_Model'
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

In [ ]:
## early stoping and chekPoint
early_stop = EarlyStopping(monitor='val_dice_coefficient', patience=7, mode='max')
filepath="weights_best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_dice_coefficient', verbose=0, save_best_only=True, mode='max')
callbacks_list = [checkpoint, early_stop,tensorboard]

In [ ]:
# Fiting the model 
results = model.fit(train_data, train_labels, batch_size=3, epochs=150, callbacks=callbacks_list,
                    validation_data=(val_data, val_labels),verbose=2)

In [ ]:
# Reading the Test data
test_data, test_labels = get_data(path_test, train=True) 
# If there is no ground-truth for test data, this line should be written test_data = get_data(path_test, train=False) 

In [ ]:
# Predict a mask for test images
preds_test = model.predict(test_data, verbose=1)
preds_test_t = (preds_test > 0.5).astype(np.uint8) # thresholding (should be between 0 and 1)

In [ ]:
# Visualization of test image and the predicted mask
fig = plt.figure(figsize=(10, 10))
plt.subplot(221)
plt.imshow(test_data[0,:,:,0], cmap=plt.get_cmap('gray'))
##### should be commented if no ground-truth for test data##### 
plt.subplot(222)
plt.imshow(test_labels[0, :, :, 0], cmap=plt.get_cmap('gray'))
########################################################
plt.subplot(223)
plt.imshow(preds_test[0, :, :, 0], cmap=plt.get_cmap('gray'))
plt.subplot(224)
plt.imshow(preds_test_t[0, :, :, 0], cmap=plt.get_cmap('gray'))
# show the plot
plt.show()